# 1_Measure_rawintensities_Ifit1

In [1]:

include("../../Code/TSSs/src/TSSs.jl")
using ProgressMeter
using DataFrames
using Distances
using CSV
using RCall

ENV["Code"] = "../../Code"
for folder in readdir(ENV["Code"]); push!(LOAD_PATH, normpath(ENV["Code"], folder));end

using CSV
using NoLongerProblems_FileHandling
using NoLongerProblems
using NoLongerProblems_Pandas
using DataFrames
using FQfiles
using HypothesisTests
using MultipleTesting
import Pandas

using PyPlot
using PrettyPlotting


function add_probetypes(e)
    col = Symbol("Probe (type)")
    type1 = []; type4 = []; type6 = []; 
    
    for ii in e[!,col]
        probes = split(replace(ii, " " => ""), ",")
        probes_pretty = [split(jj, "(")[1] for jj in probes]
        
        t1_ind = findall(x -> occursin("(1)", x), probes)
        t4_ind = findall(x -> occursin("(4)", x), probes) 
        t6_ind = findall(x -> occursin("(6)", x), probes) 
        
        t1 = if length(t1_ind) == 1 probes_pretty[t1_ind][1] else "NA" end; 
        t4 = if length(t4_ind) == 1 probes_pretty[t4_ind][1] else "NA" end; 
        t6 = if length(t6_ind) == 1 probes_pretty[t6_ind][1] else "NA" end; 
        
        push!(type1, t1); push!(type4, t4); push!(type6, t6); 
    end
    e[!,:type1] = type1; e[!,:type4] = type4; e[!,:type6] = type6; 
    e
    
end

function CellInfo(dir, exp_df)
    image_file = get_files_ending_with(dir, "Image.csv")
     cell_file = get_files_ending_with(dir, "TrueCells.csv")

    if .&(length(cell_file) == 1,  length(image_file) == 1) # Make sure that there is only one file with the image information
        image_file = normpath(dir, image_file[1])
        cell_file = normpath(dir, cell_file[1])
    else
        error("Image.csv or Cell file not found")
    end
    
    ima = DataFrames.DataFrame(CSV.read(image_file, DataFrames.DataFrame))[!, [:FileName_DAPI, :ImageNumber]]
    ima[!,:Image] = [split(split(ii, "_C1")[1], "_MAX")[1] for ii in ima[!,:FileName_DAPI]]
    
    ima[!,:WELL] = [split(ii, " ") for ii in ima[!,:Image]]
    ima[!,:WELL] = [1 for ii in ima[!,:WELL]]
    
     ima[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in ima[!,:FileName_DAPI]]
    
    exp_df[!,:Well] = [split(ii, " (")[1] for ii in exp_df[!,:Well]]

    im_ = innerjoin(exp_df, ima, on = :Well, makeunique=true)
        

    
   
     cells = DataFrames.DataFrame(CSV.read(cell_file, DataFrames.DataFrame))
    cells = innerjoin(im_, cells, on = :ImageNumber)
    
    cells[!,:Image_Cell] = [cells[ii, :Image]*"__Cell_CP_"*string(cells[ii, :ObjectNumber]) for ii in 1:nrow(cells)]
    return dropmissing(cells)
    
end





function FQ_summary_MATURE(dir)
    file = get_files_containing(dir, "summary_MATURE")
    n = length(file)
    if n != 1
        error("ERROR: $n files found")
    end
    filename = string(dir, file[1])
    
     df = CSV.read(filename,  DataFrames.DataFrame, delim = '\t', header = 5, skipto = 6)
    rename!(df, :CELL => :Cell)
    rename!(df, :FILE => :Image)
    
    df = FQfiles.fix_image_name(df)
    df = column_fusion(df, :Image, :Cell)

end

function cells_per_sample(df)
    sams = unique(df[:Name])
    ncells = [ count(x -> x == sam, df[:Name]) for sam in sams]
    new_df = DataFrames.DataFrame(:Sample=> sams, :N_Cells=> ncells)
end


function assign_red_green_labels(REP, max_green_for_red, min_red_for_red, max_red_for_green,  min_green_for_green, nrep)
    function DrawRectangles()
        topcorner = 1
        plot([max_green_for_red, max_green_for_red], [min_red_for_red, topcorner], c = "red")
        plot([0, 0], [min_red_for_red, topcorner], c = "red")
        plot([0, max_green_for_red], [min_red_for_red, min_red_for_red], c = "red")
        plot([0, max_green_for_red], [1, 1], c = "red")

        plot([min_green_for_green, 1], [max_red_for_green, max_red_for_green], c = "green")

        plot([1, 1], [0, max_red_for_green], c = "green")

        plot([min_green_for_green, min_green_for_green], [max_red_for_green, 0], c = "green")
        plot([min_green_for_green, 1], [0, 0], c = "green")
    end
    
    
    
    
    green = REP[:Intensity_UpperQuartileIntensity_FilteredGreen]
    red = REP[:Intensity_UpperQuartileIntensity_FilteredRed]
    labels = ["NotKnown" for ii in 1:length(red)]

    for ii in 1:length(red)
        r = red[ii]; g = green[ii]

        if .&(g > min_green_for_green, r < max_red_for_green)
            labels[ii] = "Green"
        elseif .&(r > min_red_for_red, g < max_green_for_red)
            labels[ii] = "Red"
        end

    end

    REP[:Label] = labels

    redcount = count(x -> x == "Red", labels)
    println("Red = $redcount")

    gcount = count(x -> x == "Green", labels)
    println("Green = $gcount")

    REP[:UpperQuartileIntensity_Green] =  REP[:Intensity_UpperQuartileIntensity_FilteredGreen] 
    REP[:UpperQuartileIntensity_Red] =  REP[:Intensity_UpperQuartileIntensity_FilteredRed]

    y = :UpperQuartileIntensity_Red
    x = :UpperQuartileIntensity_Green
    figure(figsize = (10, 5))

    subplot(1, 2, 1)

    sam = "WT"
    df = REP[REP[!,:Cells].== sam, :]
    scatter(df[x], df[y], s = 1, label = sam, c = "blue")
    
     sam = "Rad21KO"
    df = REP[REP[!,:Cells].== sam, :]
    scatter(df[!,x], df[!,y], s = 1, label = sam, c = "orange")
    
    
    
    DrawRectangles(); pretty_axes2(); ylabel(y); xlabel(x)
    
    subplot(1, 2, 2)

    sam = "Rad21KO+WT"
    df = REP[REP[:Cells].== sam, :]
    scatter(df[x], df[y], s = 1, label = sam, c = "purple")
    
    
    DrawRectangles(); pretty_axes2(); ylabel(y); xlabel(x)
    
    return REP



end

using DataFrames

exp = CSV.read("../CellMixExperiments/IbidiChamberSlide_Exp1.csv",  DataFrames.DataFrame)
exp[!,:ChamberSlide] = [if split(ii, "(")[2][1] == '1' "UT" else "3H" end for ii in exp[!,:Well]]
exp[!,:WELL] = ["1" for ii in exp[!,:Well]]
exp

cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp1/CP_results"
cells1 = CellInfo(cp_dir, add_probetypes(exp));
cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/quick_analysis_qq/CP_results"
cells2 = CellInfo(cp_dir, add_probetypes(exp));
cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp3/CP_results"
cells3 = CellInfo(cp_dir, add_probetypes(exp));


In [2]:
function TSSs.TSS_raw_quant(t2, tss_folder, image_folder, n; xy = 0.189, zx = 0.5)
    
    images_pat = intersect(TSSs.get_image_patterns(t2), TSSs.get_image_patterns(tss_folder))
    
    p = Progress(length(images_pat), 1)

    dfs = []

    for a in 1:length(images_pat)
        next!(p)
        

        pat = images_pat[a]

        # Get all the TSS in the images
        a = TSSs.find_outline(tss_folder, pat)
        tss = TSSs.cell_tss_dict(a)
        tss = TSSs.meassure_tss(tss, imagesfolder, pat, n, xy = xy, zx = zx)
    
        df = TSSs.analysis_singlechannel(tss; image = pat,  tss_name = :TSS2)
        push!(dfs, df)

    end
    
    d = TSSs.join_in_all_common_columns(dfs...)
    return d
    
end

In [3]:
# Define where are the outlines

root1 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp1"

tss_c4 = root1 * "/Segmentation_type1/_FQ_outline/_TS_detect"
imagesfolder = root1 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)


d = TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

if !in("TSS_raw", readdir())
    mkdir("TSS_raw")
end

CSV.write("TSS_raw/Ifit1_exp1.csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 1:04:29


"TSS_raw/Ifit1_exp1.csv"

In [4]:
# Define where are the outlines

root2 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/quick_analysis_qq"

tss_c4 = root2 * "/Segmentation_C4/_FQ_outline/_TS_detect"
imagesfolder = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)


d= TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

if !in("TSS_raw", readdir())
    mkdir("TSS_raw")
end

CSV.write("TSS_raw/Ifit1_exp2.csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 0:34:24


"TSS_raw/Ifit1_exp2.csv"

In [5]:
# Define where are the outlines

root3 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp3/"

tss_c4 = root3 * "Segmentation_type1/_FQ_outline/_TS_detect"
imagesfolder = root3 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)


d= TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

if !in("TSS_raw", readdir())
    mkdir("TSS_raw")
end

CSV.write("TSS_raw/Ifit1_exp3.csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 0:44:25


"TSS_raw/Ifit1_exp3.csv"

In [8]:
if !in("TSS_avgdot", readdir())
    mkdir("TSS_avgdot")
end
root2 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/quick_analysis_qq"

tss2 = CSV.read("TSS_raw/Ifit1_exp1.csv", DataFrames.DataFrame)
tss3 = CSV.read("TSS_raw/Ifit1_exp2.csv", DataFrames.DataFrame)
tss4 = CSV.read("TSS_raw/Ifit1_exp3.csv", DataFrames.DataFrame)


dot2_r1 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root1 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 1)*255
dot3_r1 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root2 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 1)*255
dot4_r1 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root3 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 1)*255

dot2_r2 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root1 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 2)*255
dot3_r2 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root2 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 2)*255
dot4_r2 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root3 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 2)*255

dot2_r3 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root1 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 3)*255
dot3_r3 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root2 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 3)*255
dot4_r3 = TSSs.int_brightest_pixel( TSSs.read_tiff_as_gray(root3 * "/type1_TSS/_mRNA_AVG_ns.tif"); radious = 3)*255


tss2[!,:TSS1_r1] = tss2[!,:locus1_int1_TSS2] ./ dot2_r1
tss2[!,:TSS1_r2] = tss2[!,:locus1_int2_TSS2] ./ dot2_r2
tss2[!,:TSS1_r3] = tss2[!,:locus1_int3_TSS2] ./ dot2_r3


tss3[!,:TSS1_r1] = tss3[!,:locus1_int1_TSS2] ./ dot3_r1
tss3[!,:TSS1_r2] = tss3[!,:locus1_int2_TSS2] ./ dot3_r2
tss3[!,:TSS1_r3] = tss3[!,:locus1_int3_TSS2] ./ dot3_r3


tss4[!,:TSS1_r1] = tss4[!,:locus1_int1_TSS2] ./ dot4_r1
tss4[!,:TSS1_r2] = tss4[!,:locus1_int2_TSS2] ./ dot4_r2
tss4[!,:TSS1_r3] = tss4[!,:locus1_int3_TSS2] ./ dot4_r3


tss2[!,:TSS2_r1] = tss2[!,:locus2_int1_TSS2] ./ dot2_r1
tss2[!,:TSS2_r2] = tss2[!,:locus2_int2_TSS2] ./ dot2_r2
tss2[!,:TSS2_r3] = tss2[!,:locus2_int3_TSS2] ./ dot2_r3


tss3[!,:TSS2_r1] = tss3[!,:locus2_int1_TSS2] ./ dot3_r1
tss3[!,:TSS2_r2] = tss3[!,:locus2_int2_TSS2] ./ dot3_r2
tss3[!,:TSS2_r3] = tss3[!,:locus2_int3_TSS2] ./ dot3_r3


tss4[!,:TSS2_r1] = tss4[!,:locus2_int1_TSS2] ./ dot4_r1
tss4[!,:TSS2_r2] = tss4[!,:locus2_int2_TSS2] ./ dot4_r2
tss4[!,:TSS2_r3] = tss4[!,:locus2_int3_TSS2] ./ dot4_r3


CSV.write("TSS_avgdot/Ifit1_exp1.csv", tss2)
CSV.write("TSS_avgdot/Ifit1_exp2.csv", tss3)
CSV.write("TSS_avgdot/Ifit1_exp3.csv", tss4)


"TSS_avgdot/Ifit1_exp3.csv"

In [9]:
ENV["Code"] = "../../Code"
for folder in readdir(ENV["Code"]); push!(LOAD_PATH, normpath(ENV["Code"], folder));end

using CSV
using FQfiles
using CPBurst
using PyPlot
using Seaborn
using NoLongerProblems_Pandas
using NoLongerProblems_FileHandling

import Pandas
using FQfiles
using DataFrames
using NoLongerProblems

function add_probetypes(e)
    col = Symbol("Probe (type)")
    type1 = []; type4 = []; type6 = []; 
    
    for ii in e[!,col]
        probes = split(replace(ii, " " => ""), ",")
        probes_pretty = [split(jj, "(")[1] for jj in probes]
        
        t1_ind = findall(x -> occursin("(1)", x), probes)
        t4_ind = findall(x -> occursin("(4)", x), probes) 
        t6_ind = findall(x -> occursin("(6)", x), probes) 
        
        t1 = if length(t1_ind) == 1 probes_pretty[t1_ind][1] else "NA" end; 
        t4 = if length(t4_ind) == 1 probes_pretty[t4_ind][1] else "NA" end; 
        t6 = if length(t6_ind) == 1 probes_pretty[t6_ind][1] else "NA" end; 
        
        push!(type1, t1); push!(type4, t4); push!(type6, t6); 
    end
    e[!,:type1] = type1; e[!,:type4] = type4; e[!,:type6] = type6; 
    e
    
end

function CellInfo(dir, exp_df)
    image_file = get_files_ending_with(dir, "Image.csv")
     cell_file = get_files_ending_with(dir, "TrueCells.csv")

    if .&(length(cell_file) == 1,  length(image_file) == 1) # Make sure that there is only one file with the image information
        image_file = normpath(dir, image_file[1])
        cell_file = normpath(dir, cell_file[1])
    else
        error("Image.csv or Cell file not found")
    end
    
    ima = CSV.read(image_file, DataFrames.DataFrame)[:, [:FileName_DAPI, :ImageNumber]]
    ima[!,:Image] = [split(split(ii, "_C1")[1], "_MAX")[1] for ii in ima[!,:FileName_DAPI]]
    
    ima[!,:WELL] = [split(ii, " ") for ii in ima[!,:Image]]
    ima[!,:WELL] = [1 for ii in ima[!,:WELL]]
    
     ima[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in ima[!,:FileName_DAPI]]
    
    exp_df[!,:Well] = [split(ii, " (")[1] for ii in exp_df[!,:Well]]

    im_ = innerjoin(exp_df, ima, on = :Well, makeunique=true)
        

    
   
     cells = CSV.read(cell_file,DataFrames.DataFrame)
    cells = innerjoin(im_, cells, on = :ImageNumber)
    
    cells[!,:Image_Cell] = [cells[ii, :Image]*"__Cell_CP_"*string(cells[ii, :ObjectNumber]) for ii in 1:nrow(cells)]
    return dropmissing(cells)
    
end


function FQ_summary_MATURE(dir)
    file = get_files_containing(dir, "summary_MATURE")
    n = length(file)
    if n != 1
        error("ERROR: $n files found")
    end
    filename = string(dir, file[1])
    
     df = CSV.read(filename, DataFrames.DataFrame, delim = '\t', header = 5, skipto = 6)
    rename!(df, :CELL => :Cell)
    rename!(df, :FILE => :Image)
    
    df = FQfiles.fix_image_name(df)
    df = column_fusion(df, :Image, :Cell)

end


exp = CSV.read("IbidiChambers/IbidiChamberSlide_Ifit1_mixexps.csv", DataFrames.DataFrame)
exp[!,:ChamberSlide] = [if split(ii, "(")[2][1] == '1' "UT" else "3H" end for ii in exp[!,:Well]]
exp[!,:WELL] = ["1" for ii in exp[!,:Well]]
exp[!,:Genotype] = exp[!,:Cells]
exp[!,:Well] = [split(ii, " (")[1] for ii in exp[!,:Well]]

exp

cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp1/CP_results"
cells1 = CellInfo(cp_dir, add_probetypes(exp));
cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/quick_analysis_qq/CP_results"
cells2 = CellInfo(cp_dir, add_probetypes(exp));
cp_dir = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp3/CP_results"
cells3 = CellInfo(cp_dir, add_probetypes(exp));
exp1 = CSV.read("TSS_avgdot/Ifit1_exp1.csv", DataFrames.DataFrame)
exp1[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in exp1[!,:Image]]
exp1 = innerjoin(exp1, exp, on= "Well")
CSV.write("Ifit1/Ifit1_exp1_TSS.csv", exp1)
CSV.write("Ifit1/Ifit1_exp1_CP.csv", cells1)
exp1 = CSV.read("TSS_avgdot/Ifit1_exp2.csv", DataFrames.DataFrame)
exp1[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in exp1[!,:Image]]
exp1 = innerjoin(exp1, exp, on= "Well")
CSV.write("Ifit1/Ifit1_exp2_TSS.csv", exp1)
CSV.write("Ifit1/Ifit1_exp2_CP.csv", cells2)
exp1 = CSV.read("TSS_avgdot/Ifit1_exp3.csv", DataFrames.DataFrame)
exp1[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in exp1[!,:Image]]
exp1 = innerjoin(exp1, exp, on= "Well")
CSV.write("Ifit1/Ifit1_exp3_TSS.csv", exp1)
CSV.write("Ifit1/Ifit1_exp3_CP.csv", cells3)

fq_dir1 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp1/type1_TSS/"
fq_dir2 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/quick_analysis_qq/type1_TSS/"
fq_dir3 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp3/type1_TSS/"


fq1 = FQ_summary_MATURE(fq_dir1)
rename!(fq1, :N_thres_Total => :N_exon)
rename!(fq1, :N_thres_Nuc => :N_exon_Nuc)
fq2 = FQ_summary_MATURE(fq_dir2)
rename!(fq2, :N_thres_Total => :N_exon)
rename!(fq2, :N_thres_Nuc => :N_exon_Nuc)
fq3 = FQ_summary_MATURE(fq_dir3)
rename!(fq3, :N_thres_Total => :N_exon)
rename!(fq3, :N_thres_Nuc => :N_exon_Nuc)

CSV.write("Ifit1/Ifit1_exp1_FQ.csv", fq1)
CSV.write("Ifit1/Ifit1_exp2_FQ.csv", fq2)
CSV.write("Ifit1/Ifit1_exp3_FQ.csv", fq3)

"Ifit1/Ifit1_exp3_FQ.csv"